# Few Shot CLIP on Tweets to Predict Emoji's

<!-- - Fetch the preprocessed dataset. The dataset contains tweets as text and the label is an emoji.
- Fine tune CLIP on our dataset.
- Push the model to Huggingface Hub. -->
<!-- 
Pretrained CLIP model: https://huggingface.co/openai/clip-vit-base-patch32
Got inspiration for finteuning here: https://github.com/huggingface/transformers/tree/main/examples/pytorch/contrastive-image-text
 -->
## 1. Install Dependencies

In [1]:
from IPython import get_ipython

# you might want to restart the kernel
# coupling between torch and torchvision: https://pypi.org/project/torchvision/
get_ipython().system('pip install torchvision==0.11.1 torch==1.10.0 --quiet')
get_ipython().system('pip install transformers datasets pillow ipywidgets requests jupyter jupyter_client wandb sklearn --upgrade --quiet')


## 2. Init Variables and Tools

In [2]:
import os
import wandb

os.environ["WANDB_DISABLED"] = "false"
wandb.init(project="emoji-predictor", entity="drift-ai")

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Curren

## 3. Load Data

In [3]:
from pathlib import Path

import torch

from transformers import CLIPProcessor, CLIPModel, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

dataset = load_dataset("vincentclaes/emoji-predictor")

train_dataset = dataset["train"]
val_dataset = dataset["validation"]

# code to take a sample for testing purposes
# train_dataset = dataset["train"].select(range(32))
# val_dataset = dataset["validation"].select(range(32))

# test_dataset = dataset["test"]

column_names = train_dataset.column_names
assert "label" in column_names
assert "text" in column_names
image_column = "label"
caption_column = "text"

Using custom data configuration vincentclaes--emoji-predictor-84ee9ecf6ec78809
Reusing dataset parquet (/root/.cache/huggingface/datasets/vincentclaes___parquet/vincentclaes--emoji-predictor-84ee9ecf6ec78809/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

## 4. Generate Few Shot Dataset

In [4]:
import matplotlib.pyplot as plt

def plot(df):
    df["label"].value_counts().plot(kind = 'bar')

    
def under_sample(df, no_rows):
    print(f"No of rows: {no_rows}")
    df = df.groupby("label", as_index=False).apply(lambda x : x.sample(n=no_rows, replace=False, axis=0)).reset_index(drop=True)
    return df

_range = [1, 5, 10, 15, 20]
df_train = train_dataset.to_pandas()
dataset_dict = {}

for i in _range:
    df_train_few_shot = under_sample(df=df_train, no_rows=i)
    dataset_dict[i] =  Dataset.from_pandas(df_train_few_shot)

dataset_dict

No of rows: 1
No of rows: 5
No of rows: 10
No of rows: 15
No of rows: 20


{1: Dataset({
     features: ['text', 'label'],
     num_rows: 20
 }),
 5: Dataset({
     features: ['text', 'label'],
     num_rows: 100
 }),
 10: Dataset({
     features: ['text', 'label'],
     num_rows: 200
 }),
 15: Dataset({
     features: ['text', 'label'],
     num_rows: 300
 }),
 20: Dataset({
     features: ['text', 'label'],
     num_rows: 400
 })}

In [7]:
for few_shots, train_dataset in dataset_dict.items():

    # fetch a freshc validation dataset
    val_dataset = dataset["validation"]

    # Load Pretrained Model and Processor.
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    config = model.config
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    tokenizer = processor.tokenizer
    feature_extractor = processor.feature_extractor

    MAX_TEXT_LENGTH = 77
    IMAGE_SIZE = config.vision_config.image_size
    
    # Process the Tweets.
    def tokenize_captions(examples):
        captions = [caption for caption in examples[caption_column]]
        text_inputs = tokenizer(captions, max_length=MAX_TEXT_LENGTH, padding="max_length", truncation=True)
        examples["input_ids"] = text_inputs.input_ids
        examples["attention_mask"] = text_inputs.attention_mask
        return examples


    train_dataset = train_dataset.map(
        function=tokenize_captions,
        batched=True,
        remove_columns=[col for col in column_names if col != image_column],
        num_proc=None,
        load_from_cache_file=False,
        desc="Running tokenizer on train dataset",
    )

    val_dataset = val_dataset.map(
        function=tokenize_captions,
        batched=True,
        remove_columns=[col for col in column_names if col != image_column],
        num_proc=None,
        load_from_cache_file=False,
        desc="Running tokenizer on val dataset",
    )

#     test_dataset = test_dataset.map(
#         function=tokenize_captions,
#         batched=True,
#         remove_columns=[col for col in column_names if col != image_column],
#         num_proc=None,
#         load_from_cache_file=False,
#         desc="Running tokenizer on test dataset",
#     )
    
    # Process the Emoji images.
    from PIL import Image

    def transform_images(examples):
        # https://pytorch.org/vision/stable/_modules/torchvision/io/image.html#ImageReadMode
        images = [Image.open(str(Path("./emojis",f"{c}.png"))) for c in examples[image_column]]
        images_transformed = processor.feature_extractor(images, return_tensors="pt")
        examples["pixel_values"] = images_transformed["pixel_values"]
        return examples


    train_dataset.set_transform(transform_images)
    val_dataset.set_transform(transform_images)
#     test_dataset.set_transform(transform_images)


    def collate_fn(examples):
        pixel_values = torch.stack([example["pixel_values"] for example in examples])
        input_ids = torch.tensor([example["input_ids"] for example in examples], dtype=torch.long)
        attention_mask = torch.tensor([example["attention_mask"] for example in examples], dtype=torch.long)
        return {
            "pixel_values": pixel_values,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "return_loss": True,
        }

    
    from datasets import load_metric
    metric = load_metric("precision")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        return metric.compute(predictions=predictions, references=labels)

    from pathlib import Path
    dir_ = f"./checkpoints-{few_shots}"
    Path(dir_).mkdir(parents=True, exist_ok=True)

    trainer = Trainer(
        model=model,
        args=TrainingArguments(output_dir=dir_,
                               dataloader_num_workers=0,
                               # number of emoji's are 20
                               per_device_eval_batch_size=20,
                               per_device_train_batch_size=20,
                               num_train_epochs=1,
                               # I couldn't make evaluation work.
                               # evaluation_strategy = "steps",
                               # eval_steps=8,
                               warmup_steps=0,
                               learning_rate=5e-05,
                               weight_decay=0.1,
                               report_to="wandb",
                               logging_strategy="epoch",
#                                logging_steps=20*few_shots,
                               save_strategy="epoch"
                               ),
        train_dataset=train_dataset,
        eval_dataset=train_dataset,
        compute_metrics=compute_metrics,
        data_collator=collate_fn,
        tokenizer=processor
    )
    
    train_result = trainer.train()

loading configuration file https://huggingface.co/openai/clip-vit-base-patch32/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3b789e359eb706935804a4c9db0b23519332fa3e1fad47b153db4fa81203a261.c3125bd3ca741b2a0408d08876a67613c987dd7431bfc56cc1a7625641ead5d8
text_config_dict is None. Initializing the CLIPTextConfig with default values.
vision_config_dict is None. initializing the CLIPVisionConfig with default values.
Model config CLIPConfig {
  "_name_or_path": "openai/clip-vit-base-patch32",
  "architectures": [
    "CLIPModel"
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 512,
  "text_config": {
    "_name_or_path": "",
    "add_cross_attention": false,
    "architectures": null,
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty"

Running tokenizer on train dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on val dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20
  Num Epochs = 1
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 1
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
1,3.399300


...

Running tokenizer on train dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on val dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 400
  Num Epochs = 1
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 20
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/opt/conda/lib/python3.8/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Step,Training Loss
20,3.096000


Saving model checkpoint to ./checkpoints-20/checkpoint-20
Configuration saved in ./checkpoints-20/checkpoint-20/config.json
Model weights saved in ./checkpoints-20/checkpoint-20/pytorch_model.bin
Feature extractor saved in ./checkpoints-20/checkpoint-20/preprocessor_config.json
tokenizer config file saved in ./checkpoints-20/checkpoint-20/tokenizer_config.json
Special tokens file saved in ./checkpoints-20/checkpoint-20/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


